## Tutorial 1: Academy Weaviate
### [101T Work with: Text Data](https://docs.weaviate.io/academy/py/starter_text_data)

### 👉 LLMs and Weaviate (RAG)
Use AI (large language models) to augment and transform retrieved data.

- Describe what RAG is and how it works at a high level.
- Perform RAG on individual objects.
- Perform RAG on the entire set of returned objects.

## ➡️🧠🤓 RAG: Overview
`Retrieval augmented generation` (**RAG**) is a way to <u>combine</u> the best of both worlds: <u>the retrieval capabilities of **semantic search** and the **generation** capabilities of AI models</u> such as **large language models**. This allows you to <u>retrieve objects from a Weaviate instance and then generate outputs based on the retrieved objects</u>.

When we created a collection, we specified the `generative_module` parameter as shown here.
```bash
generative_config=wc.Configure.Generative.openai()

```

This selects a `generative module` that will be used to generate outputs based on the retrieved objects. 
In this case, we're using the openai module, and the GPT family of large language models.

As we did before with the `vectorizer` **module**, you will require an `API key` from the provider of the generative module.

**RAG queries** are also called **'generative' queries** in **Weaviate**. 

You can access these functions through the generate submodule of the collection object.

Each generative query works in addition to the regular search query, and will perform a RAG query on each retrieved object.

## ➡️🧠🤓 'Single prompt' generation
A 'single prompt' generation will perform RAG queries on each retrieved object. This is useful when you want to transform each object separately, with the same prompt.

This example finds entries in "Movie" whose vector best matches the query vector (for "dystopian future"). Then, instructs the large language model to translate the title of each movie into French. Each of the results is then printed out to the console.

```bash
import os
import weaviate

# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_weaviate_cloud(..., headers=headers) or
# client = weaviate.connect_to_local(..., headers=headers)

# Get the collection
movies = client.collections.use("Movie")

# Perform query
response = movies.generate.near_text(
    query="dystopian future",
    limit=5,
    single_prompt="Translate this into French: {title}"
)

# Inspect the response
for o in response.objects:
    print(o.properties["title"])  # Print the title
    print(o.generated)  # Print the generated text (the title, in French)

client.close()
````

## ◻️ Explain the code
You must pass on <u>one or more properties</u> to the `single_prompt` parameter through **braces**, as we've done here with "... {title} ...". <u>This will instruct Weaviate to pass on the `title` property from each retrieved object to the large language model.</u>

### ◽Response object
Each response object is similar to that from a regular search query, with an additional generated attribute. This attribute will contain the generated output for each object.

## ➡️🧠🤓 'Grouped task' generation

A 'grouped task' generation will perform RAG queries on the set of retrieved objects. This is useful when you want to transform the set of objects as a whole, with one prompt.

This example <u>finds entries in "Movie" whose vector best matches the query vector</u> (for "dystopian future"). Then, instructs the large language model to find commonalities between them.

```bash
import os
import weaviate

# Instantiate your client (not shown). e.g.:
# headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}  # Replace with your OpenAI API key
# client = weaviate.connect_to_weaviate_cloud(..., headers=headers) or
# client = weaviate.connect_to_local(..., headers=headers)

# Get the collection
movies = client.collections.use("Movie")

# Perform query
response = movies.generate.near_text(
    query="dystopian future",
    limit=5,
    grouped_task="What do these movies have in common?",
    # grouped_properties=["title", "overview"]  # Optional parameter; for reducing prompt length
)

# Inspect the response
for o in response.objects:
    print(o.properties["title"])  # Print the title
print(response.generative.text)  # Print the generated text (the commonalities between them)

client.close()

```

## ◻️ Explain the code
For grouped_task queries, you simply pass on the prompt to the `grouped_task` parameter. This will instruct Weaviate to pass on the:

- text properties from all retrieved objects, and the prompt to the large language model.

Example results:
```bash
In Time
Looper
I, Robot
The Matrix
Children of Men
These movies all involve futuristic settings and explore themes related to the manipulation of time, technology, and the potential consequences of advancements in society. They also touch on issues such as inequality, control, and the impact of human actions on the future of humanity.
```

### ◽Response object
A RAG query with the grouped_task parameter will return a response with an additional generated attribute. This attribute will contain the generated output for the set of objects.